<a href="https://colab.research.google.com/github/ElaYJ/Study_Deep_Learning/blob/main/Lecture/11_PyTorch_CNN_Plant_Leaf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 식물잎의 사진으로 질병 분류

- 이 자료는 병든 작물잎 사진을 이용해 질병의 유무를 판단하는 분류문제로 kaggle에서 좋은 평가를 받은 코드를 가지고 해설한다.

<br></br>

## - 파일 정리

- https://drive.google.com/file/d/1tzhqc3384i5bd_OIX95ACYkaB44ZpxKj/view?usp=sharing

- 데이터를 받아서 구글 드라이브에 올리고 압축을 풀어서 나의 colab 폴더에 둔다.

- 4만 장 사진 데이터 (balance가 잘 맞지는 않는다.)

    <img src="https://github.com/ElaYJ/supplement/assets/153154981/d4276a45-db5e-4781-b23b-5f24f2f1178b" width="57%">

In [1]:
!cat /proc/uptime | awk '{printf("잔여시간 : %.2f", 12-$1/60/60)}'

잔여시간 : 11.88

In [2]:
!nvidia-smi

Fri Apr 12 16:37:43 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [9]:
# 구글 드라이브에 접근

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [11]:
# -d --> 압축을 푸는 경로 지정

!unzip -qq "/content/drive/MyDrive/zero-base DS/dataset.zip" -d './dataset'

In [12]:
import os

original_dataset_dir = './dataset'
classes_list = os.listdir(original_dataset_dir)
classes_list

['Tomato___healthy',
 'Tomato___Late_blight',
 'Tomato___Tomato_Yellow_Leaf_Curl_Virus',
 'Pepper,_bell___healthy',
 'Potato___Early_blight',
 'Strawberry___Leaf_scorch',
 'Grape___healthy',
 'Tomato___Bacterial_spot',
 'Cherry___Powdery_mildew',
 'Corn___Common_rust',
 'Tomato___Early_blight',
 'Grape___Leaf_blight_(Isariopsis_Leaf_Spot)',
 'Tomato___Tomato_mosaic_virus',
 'Tomato___Target_Spot',
 'Potato___healthy',
 'Potato___Late_blight',
 'Cherry___healthy',
 'Pepper,_bell___Bacterial_spot',
 'Grape___Black_rot',
 'Peach___Bacterial_spot',
 'Strawberry___healthy',
 'Corn___healthy',
 'Tomato___Leaf_Mold',
 'Apple___Black_rot',
 'Apple___Apple_scab',
 'Tomato___Spider_mites Two-spotted_spider_mite',
 'Corn___Northern_Leaf_Blight',
 'Grape___Esca_(Black_Measles)',
 'Tomato___Septoria_leaf_spot',
 'Apple___healthy',
 'Apple___Cedar_apple_rust',
 'Corn___Cercospora_leaf_spot Gray_leaf_spot',
 'Peach___healthy']

In [13]:
base_dir = './splitted'
os.mkdir(base_dir)

In [14]:
# 데이터 정리를 위한 목록 및 폴더 생성

train_dir = os.path.join(base_dir, 'train')
os.mkdir(train_dir)

validation_dir = os.path.join(base_dir, 'valid')
os.mkdir(validation_dir)

test_dir = os.path.join(base_dir, 'test')
os.mkdir(test_dir)

for cls in classes_list:
    os.mkdir(os.path.join(train_dir, cls))
    os.mkdir(os.path.join(validation_dir, cls))
    os.mkdir(os.path.join(test_dir, cls))

In [15]:
import math
import shutil

# 데이터 현황 확인
for cls in classes_list:
    path = os.path.join(original_dataset_dir, cls)
    fnames = os.listdir(path)

    train_size = math.floor(len(fnames) * 0.6)
    valid_size = math.floor(len(fnames) * 0.2)
    test_size = math.floor(len(fnames) * 0.2)

    train_fnames = fnames[:train_size]
    print("Train size(",cls,")", len(train_fnames))
    for fname in train_fnames:
        src = os.path.join(path, fname)
        dst = os.path.join(os.path.join(train_dir, cls), fname)
        shutil.copyfile(src, dst)

    valid_fnames = fnames[train_size:(train_size + valid_size)]
    print("Valid size(",cls,")", len(valid_fnames))
    for fname in valid_fnames:
        src = os.path.join(path, fname)
        dst = os.path.join(os.path.join(validation_dir, cls), fname)
        shutil.copyfile(src, dst)

    test_fnames = fnames[(train_size + valid_size):(train_size + valid_size + test_size)]
    print("Test size(",cls,")", len(test_fnames))
    for fname in test_fnames:
        src = os.path.join(path, fname)
        dst = os.path.join(os.path.join(test_dir, cls), fname)
        shutil.copyfile(src, dst)

Train size( Tomato___healthy ) 954
Valid size( Tomato___healthy ) 318
Test size( Tomato___healthy ) 318
Train size( Tomato___Late_blight ) 1145
Valid size( Tomato___Late_blight ) 381
Test size( Tomato___Late_blight ) 381
Train size( Tomato___Tomato_Yellow_Leaf_Curl_Virus ) 3214
Valid size( Tomato___Tomato_Yellow_Leaf_Curl_Virus ) 1071
Test size( Tomato___Tomato_Yellow_Leaf_Curl_Virus ) 1071
Train size( Pepper,_bell___healthy ) 886
Valid size( Pepper,_bell___healthy ) 295
Test size( Pepper,_bell___healthy ) 295
Train size( Potato___Early_blight ) 600
Valid size( Potato___Early_blight ) 200
Test size( Potato___Early_blight ) 200
Train size( Strawberry___Leaf_scorch ) 665
Valid size( Strawberry___Leaf_scorch ) 221
Test size( Strawberry___Leaf_scorch ) 221
Train size( Grape___healthy ) 253
Valid size( Grape___healthy ) 84
Test size( Grape___healthy ) 84
Train size( Tomato___Bacterial_spot ) 1276
Valid size( Tomato___Bacterial_spot ) 425
Test size( Tomato___Bacterial_spot ) 425
Train size( 

<br></br>

## - 모델 학습

In [16]:
import torch

USE_CUDA = torch.cuda.is_available()
DEVICE = torch.device("cuda" if USE_CUDA else "CPU")
print(DEVICE)

batch_size = 256
epochs = 30

cuda


In [17]:
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder

transform_base = transforms.Compose([transforms.Resize((64,64)), transforms.ToTensor()])
train_dataset = ImageFolder(root='./splitted/train', transform=transform_base)
valid_dataset = ImageFolder(root='./splitted/valid', transform=transform_base)

In [18]:
from torch.utils.data import DataLoader

train_loader = torch.utils.data.DataLoader(
    train_dataset, batch_size=batch_size, shuffle=True, num_workers=4
)
valid_loader = torch.utils.data.DataLoader(
    valid_dataset, batch_size=batch_size, shuffle=True, num_workers=4
)

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [19]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

class Net(nn.Module):

    def __init__(self):

        super(Net, self).__init__()

        self.conv1 = nn.Conv2d(3, 32, 3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        self.conv3 = nn.Conv2d(64, 64, 3, padding=1)
        self.pool = nn.MaxPool2d(2,2)
        self.fc1 = nn.Linear(4096, 512)
        self.fc2 = nn.Linear(512, 33) #--> class가 33개임.

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.pool(x)
        x = F.dropout(x, p=0.25, training=self.training) #--> train data만 적용

        x = self.conv2(x)
        x = F.relu(x)
        x = self.pool(x)
        x = F.dropout(x, p=0.25, training=self.training)

        x = self.conv3(x)
        x = F.relu(x)
        x = self.pool(x)
        x = F.dropout(x, p=0.25, training=self.training)

        x = x.view(-1, 4096) #--> Flatten과 동일
        x = self.fc1(x)
        x = F.relu(x)
        x = F.dropout(x, p=0.25, training=self.training)
        x = self.fc2(x)

        return F.log_softmax(x, dim=1)


In [20]:
model_base = Net().to(DEVICE)
optimizer = optim.Adam(model_base.parameters(), lr=0.001)

In [21]:
def train(model, train_loader, optimizer):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(DEVICE), target.to(DEVICE) #--> data, target을 Device로 보낸다.
        optimizer.zero_grad()
        output = model(data)
        loss = F.cross_entropy(output, target)
        loss.backward() #--> 역전파
        optimizer.step() #--> weight update

In [22]:
def evaluate(model, test_loader):
    model.eval()
    test_loss = 0
    correct = 0

    with torch.no_grad(): #--> 모델을 평가할 때는 가중치를 업데이트하면 안된다.
        for data, target in test_loader:
            data, target = data.to(DEVICE), target.to(DEVICE)
            output = model(data)

            test_loss += F.cross_entropy(output, target, reduction='sum').item()

            pred = output.max(1, keepdim=True)[1]
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    test_accuracy = 100 * correct / len(test_loader.dataset)
    return test_loss, test_accuracy

In [23]:
import time
import copy

def train_baseline(model, train_loader, valid_loader, optimizer, num_epochs = 30):
    best_acc = 0.0
    best_model_wts = copy.deepcopy(model.state_dict()) #--> 정확도가 가장 높은 모델의 가중치 저장

    for epoch in range(1, num_epochs+1):
        since = time.time()
        train(model, train_loader, optimizer)
        train_loss, train_acc = evaluate(model, train_loader)
        valid_loss, valid_acc = evaluate(model, valid_loader)

        # 30번째 epoch의 모델이 Best 모델이라는 보장이 없다.
        if valid_acc > best_acc:
            best_acc = valid_acc
            best_model_wts = copy.deepcopy(model.state_dict())

        time_elapsed = time.time() - since
        print('------------------ epoch {} ------------------'.format(epoch))
        print('train Loss: {:.4f}, Accuracy: {:.2f}%'.format(train_loss, train_acc))
        print('valid Loss: {:.4f}, Accuracy: {:.2f}%'.format(valid_loss, valid_acc))
        print('Completed in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))

    model.load_state_dict(best_model_wts)
    return model

In [24]:
base = train_baseline(model_base, train_loader, valid_loader, optimizer, epochs)

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


------------------ epoch 1 ------------------
train Loss: 1.5231, Accuracy: 56.38%
valid Loss: 1.5465, Accuracy: 55.49%
Completed in 1.000000m 7s
------------------ epoch 2 ------------------
train Loss: 0.9729, Accuracy: 69.79%
valid Loss: 1.0073, Accuracy: 68.67%
Completed in 1.000000m 1s
------------------ epoch 3 ------------------
train Loss: 0.7857, Accuracy: 74.67%
valid Loss: 0.8400, Accuracy: 73.24%
Completed in 1.000000m 2s
------------------ epoch 4 ------------------
train Loss: 0.6929, Accuracy: 77.78%
valid Loss: 0.7522, Accuracy: 75.94%
Completed in 1.000000m 2s
------------------ epoch 5 ------------------
train Loss: 0.4559, Accuracy: 85.41%
valid Loss: 0.5243, Accuracy: 82.68%
Completed in 1.000000m 1s
------------------ epoch 6 ------------------
train Loss: 0.4255, Accuracy: 86.60%
valid Loss: 0.5070, Accuracy: 83.98%
Completed in 1.000000m 1s
------------------ epoch 7 ------------------
train Loss: 0.3362, Accuracy: 89.50%
valid Loss: 0.4275, Accuracy: 86.29%
Comp

In [25]:
torch.save(base, 'baseline.pt')